# Forecasting Model Development
## Multi-Agent Economic Forecasting System - Notebook 3

**Objective**: Develop and evaluate forecasting models for economic indicators using statistical methods and the Forecasting Specialist agent.

### What You'll Learn:
- ARIMA model implementation and hyperparameter tuning
- Exponential smoothing methods for time series forecasting
- Ensemble forecasting techniques for improved accuracy
- Model performance evaluation and comparison
- Using the Forecasting Specialist agent for automated forecasting

## 1. Setup and Data Preparation

In [ ]:
# Import required libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import asyncio
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

# Import forecasting components
from tools.statistical_tools import StatisticalTools
from agents.forecasting_specialist import ForecastingSpecialistAgent
from google.adk.models.google_llm import Gemini
from google.genai import types

# Setup visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Forecasting libraries and components imported successfully")

In [ ]:
# Initialize forecasting components
stat_tools = StatisticalTools()

# Initialize model for agents
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

model = Gemini(
    model="gemini-2.0-flash-exp",
    retry_options=retry_config
)

# Initialize Forecasting Specialist Agent
forecasting_agent = ForecastingSpecialistAgent(model)

print("🤖 Forecasting Specialist Agent initialized")
print("🛠️ Available forecasting tools:")
for tool in forecasting_agent.agent.tools:
    print(f"   - {tool.name}")

## 2. Load and Prepare Forecasting Data

In [ ]:
# Generate realistic economic data for forecasting
def generate_forecasting_data():
    """Generate sample economic data with clear patterns for forecasting"""
    dates = pd.date_range(start='2000-01-01', end='2024-12-31', freq='Q')
    np.random.seed(42)

    # Create more realistic economic data
    n_periods = len(dates)

    # Long-term trend
    trend = np.linspace(80, 200, n_periods)

    # Seasonal component (quarterly patterns)
    seasonal = 15 * np.sin(2 * np.pi * np.arange(n_periods) / 4)

    # Business cycles (multiple frequencies)
    cycle1 = 10 * np.sin(2 * np.pi * np.arange(n_periods) / 32)  # 8-year cycles
    cycle2 = 5 * np.sin(2 * np.pi * np.arange(n_periods) / 16)   # 4-year cycles

    # Structural breaks (economic events)
    structural_breaks = np.zeros(n_periods)
    break_points = [20, 45, 70]  # Simulate economic events
    for bp in break_points:
        if bp < n_periods:
            structural_breaks[bp:] += np.random.normal(-10, 5)

    # Noise component
    noise = np.random.normal(0, 3, n_periods)

    # Combine all components
    gdp_data = trend + seasonal + cycle1 + cycle2 + structural_breaks + noise

    return pd.DataFrame({
        'TimePeriod': dates,
        'DataValue': gdp_data,
        'Series': 'GDP'
    })

# Generate data
forecast_data = generate_forecasting_data()
print(f"📊 Forecasting data generated: {len(forecast_data)} quarters")
print(f"📅 Date range: {forecast_data['TimePeriod'].min()} to {forecast_data['TimePeriod'].max()}")

# Split into training and test sets
split_point = int(len(forecast_data) * 0.8)
train_data = forecast_data.iloc[:split_point]
test_data = forecast_data.iloc[split_point:]

print(f"\n📈 Data split for model evaluation:")
print(f"   Training: {len(train_data)} quarters ({train_data['TimePeriod'].min()} to {train_data['TimePeriod'].max()})")
print(f"   Testing:  {len(test_data)} quarters ({test_data['TimePeriod'].min()} to {test_data['TimePeriod'].max()})")

forecast_data.head()

In [ ]:
# Visualize the complete dataset
fig = go.Figure()

# Training data
fig.add_trace(go.Scatter(
    x=train_data['TimePeriod'], y=train_data['DataValue'],
    mode='lines', name='Training Data',
    line=dict(color='#1f77b4', width=3)
))

# Test data
fig.add_trace(go.Scatter(
    x=test_data['TimePeriod'], y=test_data['DataValue'],
    mode='lines', name='Test Data',
    line=dict(color='#ff7f0e', width=3)
))

# Split point
split_date = train_data['TimePeriod'].max()
fig.add_vline(x=split_date, line_dash="dash", line_color="red",
              annotation_text="Train/Test Split", annotation_position="top left")

fig.update_layout(
    title='Economic Data for Forecasting (Train/Test Split)',
    xaxis_title='Time Period',
    yaxis_title='GDP Value',
    height=500,
    showlegend=True
)

fig.show()

## 3. ARIMA Forecasting

In [ ]:
# Build and evaluate ARIMA model
print("🔮 Developing ARIMA Forecasting Model...")

# First, build the ARIMA model on training data
arima_model = stat_tools.build_arima_model(train_data, auto_select=True, max_order=3)

if arima_model['status'] == 'success':
    print("✅ ARIMA Model Built Successfully:")
    print(f"   Best Order: {arima_model.get('best_order', 'N/A')}")
    print(f"   AIC: {arima_model['summary'].get('aic', 'N/A'):.2f}")
    print(f"   BIC: {arima_model['summary'].get('bic', 'N/A'):.2f}")
    print(f"   Log-Likelihood: {arima_model['summary'].get('log_likelihood', 'N/A'):.2f}")

    # Show key parameters
    print(f"\n📊 Model Parameters:")
    params = arima_model.get('parameters', {})
    for param, value in list(params.items())[:5]:  # Show first 5 parameters
        print(f"   {param}: {value:.4f}")

    # Model accuracy
    accuracy = arima_model.get('forecast_accuracy', {})
    print(f"\n🎯 Training Accuracy:")
    print(f"   MAE: {accuracy.get('mae', 'N/A'):.2f}")
    print(f"   RMSE: {accuracy.get('rmse', 'N/A'):.2f}")
    print(f"   MAPE: {accuracy.get('mape', 'N/A'):.2f}%")
    print(f"   R²: {accuracy.get('r2', 'N/A'):.3f}")
else:
    print("❌ ARIMA model building failed")
    print(arima_model)

In [ ]:
# Generate forecasts using ARIMA
print("\n📈 Generating ARIMA Forecasts...")
arima_forecast = stat_tools.forecast_arima(train_data, periods=len(test_data))

if arima_forecast['status'] == 'success':
    print("✅ ARIMA Forecasts Generated:")
    print(f"   Forecast Periods: {len(arima_forecast['forecasts'])}")

    # Show first few forecasts
    print(f"\n🔮 Sample Forecasts:")
    for i, forecast in enumerate(arima_forecast['forecasts'][:3]):
        print(f"   Period {forecast['period_ahead']}: {forecast['point_forecast']:.2f} "
              f"(CI: {forecast['confidence_lower']:.2f} - {forecast['confidence_upper']:.2f})")

    # Calculate test accuracy
    actual_values = test_data['DataValue'].values
    predicted_values = [f['point_forecast'] for f in arima_forecast['forecasts']]

    from sklearn.metrics import mean_absolute_error, mean_squared_error
    test_mae = mean_absolute_error(actual_values, predicted_values)
    test_rmse = np.sqrt(mean_squared_error(actual_values, predicted_values))

    print(f"\n🎯 Test Set Accuracy:")
    print(f"   MAE: {test_mae:.2f}")
    print(f"   RMSE: {test_rmse:.2f}")
else:
    print("❌ ARIMA forecasting failed")
    print(arima_forecast)

In [ ]:
# Visualize ARIMA forecasts
if arima_forecast['status'] == 'success':
    fig = go.Figure()

    # Training data
    fig.add_trace(go.Scatter(
        x=train_data['TimePeriod'], y=train_data['DataValue'],
        mode='lines', name='Training Data',
        line=dict(color='#1f77b4', width=3)
    ))

    # Test data (actual)
    fig.add_trace(go.Scatter(
        x=test_data['TimePeriod'], y=test_data['DataValue'],
        mode='lines', name='Actual Test Data',
        line=dict(color='#2ca02c', width=3)
    ))

    # Forecasts
    forecast_dates = pd.to_datetime([f['period'] for f in arima_forecast['forecasts']])
    forecast_values = [f['point_forecast'] for f in arima_forecast['forecasts']]
    confidence_lower = [f['confidence_lower'] for f in arima_forecast['forecasts']]
    confidence_upper = [f['confidence_upper'] for f in arima_forecast['forecasts']]

    fig.add_trace(go.Scatter(
        x=forecast_dates, y=forecast_values,
        mode='lines', name='ARIMA Forecast',
        line=dict(color='#ff7f0e', width=3, dash='dash')
    ))

    # Confidence interval
    fig.add_trace(go.Scatter(
        x=forecast_dates + forecast_dates[::-1],
        y=confidence_upper + confidence_lower[::-1],
        fill='toself',
        fillcolor='rgba(255, 127, 14, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        name='95% Confidence Interval'
    ))

    fig.update_layout(
        title='ARIMA Model Forecast vs Actual',
        xaxis_title='Time Period',
        yaxis_title='GDP Value',
        height=500,
        showlegend=True
    )

    fig.show()
else:
    print("❌ Cannot visualize forecasts - ARIMA forecasting failed")

## 4. Ensemble Forecasting

In [ ]:
# Generate ensemble forecast combining multiple methods
print("🔄 Developing Ensemble Forecast...")
ensemble_forecast = stat_tools.ensemble_forecast(train_data, periods=len(test_data))

if ensemble_forecast['status'] == 'success':
    print("✅ Ensemble Forecast Generated:")
    print(f"   Methods used: {', '.join(ensemble_forecast['methods_used'])}")

    # Show model weights
    print(f"\n⚖️ Model Weights:")
    for method, weight in ensemble_forecast['weights'].items():
        print(f"   {method}: {weight:.3f}")

    # Calculate ensemble accuracy
    ensemble_predictions = ensemble_forecast['ensemble_forecast']
    ensemble_mae = mean_absolute_error(actual_values, ensemble_predictions)
    ensemble_rmse = np.sqrt(mean_squared_error(actual_values, ensemble_predictions))

    print(f"\n🎯 Ensemble Test Accuracy:")
    print(f"   MAE: {ensemble_mae:.2f}")
    print(f"   RMSE: {ensemble_rmse:.2f}")

    # Compare with individual methods
    if arima_forecast['status'] == 'success':
        improvement_mae = ((test_mae - ensemble_mae) / test_mae) * 100
        improvement_rmse = ((test_rmse - ensemble_rmse) / test_rmse) * 100

        print(f"\n📊 Improvement vs ARIMA:")
        print(f"   MAE improvement: {improvement_mae:.1f}%")
        print(f"   RMSE improvement: {improvement_rmse:.1f}%")
else:
    print("❌ Ensemble forecasting failed")
    print(ensemble_forecast)

In [ ]:
# Visualize ensemble forecast comparison
if (arima_forecast['status'] == 'success' and
    ensemble_forecast['status'] == 'success'):

    fig = go.Figure()

    # Training data
    fig.add_trace(go.Scatter(
        x=train_data['TimePeriod'], y=train_data['DataValue'],
        mode='lines', name='Training Data',
        line=dict(color='#1f77b4', width=2),
        opacity=0.7
    ))

    # Test data (actual)
    fig.add_trace(go.Scatter(
        x=test_data['TimePeriod'], y=test_data['DataValue'],
        mode='lines', name='Actual',
        line=dict(color='#2ca02c', width=4)
    ))

    # ARIMA forecast
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=forecast_values,
        mode='lines', name='ARIMA Forecast',
        line=dict(color='#ff7f0e', width=3, dash='dash')
    ))

    # Ensemble forecast
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=ensemble_predictions,
        mode='lines', name='Ensemble Forecast',
        line=dict(color='#d62728', width=3, dash='dot')
    ))

    fig.update_layout(
        title='Forecast Method Comparison: ARIMA vs Ensemble',
        xaxis_title='Time Period',
        yaxis_title='GDP Value',
        height=500,
        showlegend=True
    )

    fig.show()
else:
    print("❌ Cannot visualize forecast comparison")

## 5. Model Performance Evaluation

In [ ]:
# Comprehensive model evaluation
def evaluate_forecast_models(actual, predictions_dict):
    """Evaluate multiple forecasting models"""

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    results = {}

    for model_name, predictions in predictions_dict.items():
        if len(predictions) == len(actual):
            mae = mean_absolute_error(actual, predictions)
            rmse = np.sqrt(mean_squared_error(actual, predictions))
            mape = np.mean(np.abs((actual - predictions) / actual)) * 100
            r2 = r2_score(actual, predictions)

            results[model_name] = {
                'MAE': mae,
                'RMSE': rmse,
                'MAPE': mape,
                'R2': r2
            }

    return results

# Collect predictions from all models
all_predictions = {}

if arima_forecast['status'] == 'success':
    all_predictions['ARIMA'] = [f['point_forecast'] for f in arima_forecast['forecasts']]

if ensemble_forecast['status'] == 'success':
    all_predictions['Ensemble'] = ensemble_forecast['ensemble_forecast']

# Add simple baseline models
# 1. Naive forecast (last value)
naive_forecast = [train_data['DataValue'].iloc[-1]] * len(test_data)
all_predictions['Naive'] = naive_forecast

# 2. Moving average
ma_forecast = [train_data['DataValue'].tail(4).mean()] * len(test_data)
all_predictions['Moving Average'] = ma_forecast

# Evaluate all models
if all_predictions:
    evaluation_results = evaluate_forecast_models(actual_values, all_predictions)

    print("📊 FORECAST MODEL COMPARISON")
    print("=" * 60)

    # Create comparison table
    comparison_data = []
    for model_name, metrics in evaluation_results.items():
        comparison_data.append([
            model_name,
            f"{metrics['MAE']:.2f}",
            f"{metrics['RMSE']:.2f}",
            f"{metrics['MAPE']:.2f}%",
            f"{metrics['R2']:.3f}"
        ])

    comparison_df = pd.DataFrame(comparison_data,
                                columns=['Model', 'MAE', 'RMSE', 'MAPE', 'R²'])

    print(comparison_df.to_string(index=False))

    # Identify best model
    best_model = min(evaluation_results.items(), key=lambda x: x[1]['RMSE'])
    print(f"\n🏆 Best Model: {best_model[0]} (RMSE: {best_model[1]['RMSE']:.2f})")
else:
    print("❌ No predictions available for evaluation")

In [ ]:
# Visualize model performance comparison
if evaluation_results:
    models = list(evaluation_results.keys())
    rmse_values = [evaluation_results[model]['RMSE'] for model in models]
    mae_values = [evaluation_results[model]['MAE'] for model in models]

    fig = make_subplots(rows=1, cols=2,
                       subplot_titles=('RMSE Comparison', 'MAE Comparison'))

    # RMSE plot
    fig.add_trace(
        go.Bar(x=models, y=rmse_values, name='RMSE',
               marker_color=['#1f77b4' if model != best_model[0] else '#ff7f0e'
                           for model in models]),
        row=1, col=1
    )

    # MAE plot
    fig.add_trace(
        go.Bar(x=models, y=mae_values, name='MAE',
               marker_color=['#1f77b4' if model != best_model[0] else '#ff7f0e'
                           for model in models]),
        row=1, col=2
    )

    fig.update_layout(
        title_text="Forecast Model Performance Comparison",
        showlegend=False,
        height=400
    )

    fig.show()
else:
    print("❌ No evaluation results to visualize")

## 6. Using Forecasting Specialist Agent

In [ ]:
# Test the Forecasting Specialist Agent
async def test_forecasting_agent():
    """Test the forecasting specialist agent's capabilities"""

    print("🧪 Testing Forecasting Specialist Agent...")

    # Convert data to list of dictionaries for the agent
    train_data_dict = train_data.to_dict('records')

    # Test GDP forecasting
    print("\n🔸 Testing GDP forecasting...")
    forecast_result = await forecasting_agent.forecast_gdp(train_data_dict, horizon=8)
    if forecast_result['status'] == 'success':
        print(f"   Forecast horizon: {forecast_result.get('horizon', 'N/A')} periods")
        print(f"   Next quarter prediction: {forecast_result.get('next_quarter_prediction', 'N/A')}")
        print(f"   Confidence: {forecast_result.get('confidence', 0):.2f}")

    # Test ARIMA model building
    print("\n🔸 Testing ARIMA model building...")
    arima_result = await forecasting_agent.build_arima_model(train_data_dict)
    if arima_result['status'] == 'success':
        print(f"   AIC: {arima_result.get('aic', 'N/A'):.2f}")
        print(f"   Model built successfully")

    # Test ensemble forecasting
    print("\n🔸 Testing ensemble forecasting...")
    ensemble_result = await forecasting_agent.generate_ensemble_forecast(train_data_dict)
    if ensemble_result['status'] == 'success':
        print(f"   Model weights: {ensemble_result.get('model_weights', {})}")
        print(f"   Combined prediction calculated")

# Run the agent tests
await test_forecasting_agent()

## 7. Future Forecast Generation

In [ ]:
# Generate future forecasts using the best model
print("🔮 Generating Future Forecasts...")

future_periods = 12  # Forecast 3 years into the future

# Use ensemble method for future forecasts
future_forecast = stat_tools.ensemble_forecast(forecast_data, periods=future_periods)

if future_forecast['status'] == 'success':
    print("✅ Future Forecasts Generated")

    # Create future dates
    last_date = forecast_data['TimePeriod'].max()
    future_dates = pd.date_range(start=last_date + pd.DateOffset(months=3),
                                periods=future_periods, freq='Q')

    print(f"\n📅 Forecast Period: {future_dates[0].strftime('%Y-%m')} to {future_dates[-1].strftime('%Y-%m')}")

    # Display key forecasts
    print(f"\n🔮 Key Future Predictions:")
    for i, (date, prediction) in enumerate(zip(future_dates[:6], future_forecast['ensemble_forecast'][:6])):
        print(f"   {date.strftime('%Y-%m')}: {prediction:.2f}")

    # Calculate growth projections
    current_value = forecast_data['DataValue'].iloc[-1]
    one_year_growth = ((future_forecast['ensemble_forecast'][3] - current_value) / current_value) * 100
    two_year_growth = ((future_forecast['ensemble_forecast'][7] - current_value) / current_value) * 100

    print(f"\n📈 Projected Growth:")
    print(f"   1-year growth: {one_year_growth:.2f}%")
    print(f"   2-year growth: {two_year_growth:.2f}%")
else:
    print("❌ Future forecasting failed")
    print(future_forecast)

In [ ]:
# Visualize historical data with future forecasts
if future_forecast['status'] == 'success':
    fig = go.Figure()

    # Historical data
    fig.add_trace(go.Scatter(
        x=forecast_data['TimePeriod'], y=forecast_data['DataValue'],
        mode='lines', name='Historical Data',
        line=dict(color='#1f77b4', width=3)
    ))

    # Future forecasts
    fig.add_trace(go.Scatter(
        x=future_dates, y=future_forecast['ensemble_forecast'],
        mode='lines+markers', name='Future Forecast',
        line=dict(color='#ff7f0e', width=3, dash='dash'),
        marker=dict(size=6)
    ))

    # Current point
    fig.add_trace(go.Scatter(
        x=[last_date], y=[current_value],
        mode='markers', name='Current',
        marker=dict(color='red', size=10, symbol='star')
    ))

    fig.update_layout(
        title='Economic Forecast: Historical Data and Future Projections',
        xaxis_title='Time Period',
        yaxis_title='GDP Value',
        height=500,
        showlegend=True
    )

    fig.show()
else:
    print("❌ Cannot visualize future forecasts")

## 8. Summary and Next Steps

In [ ]:
print("🎯 NOTEBOOK 3 SUMMARY")
print("=" * 50)

# Summary of forecasting activities
forecasting_activities = []
if arima_forecast['status'] == 'success':
    forecasting_activities.append("ARIMA Modeling")
if ensemble_forecast['status'] == 'success':
    forecasting_activities.append("Ensemble Forecasting")
if future_forecast['status'] == 'success':
    forecasting_activities.append("Future Projections")

print(f"✅ Forecasting activities completed: {len(forecasting_activities)}")
for activity in forecasting_activities:
    print(f"   • {activity}")

print(f"\n📊 Models evaluated: {len(evaluation_results) if 'evaluation_results' in locals() else 0}")
if 'best_model' in locals():
    print(f"🏆 Best performing model: {best_model[0]}")

print(f"\n🔮 Future forecasts: {future_periods} quarters ({future_periods//4} years)")

print("\n🔜 Next Steps:")
print("   1. Proceed to Notebook 4: Multi-Agent System Demo")
print("   2. Implement real-time forecasting with live data")
print("   3. Add more sophisticated models (Prophet, LSTM)")
print("   4. Implement forecast uncertainty quantification")

print("\n💡 Production Insights:")
print("   - Ensemble methods typically outperform individual models")
print("   - Regular model retraining improves forecast accuracy")
print("   - Confidence intervals provide crucial context for decisions")
print("   - The Forecasting Specialist Agent automates complex modeling tasks")